<a href="https://colab.research.google.com/github/Miya-herman/Python-Project-1/blob/master/Python_tutorial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import libraries
import pandas as pd
import numpy as np

In [0]:
import plotly.express as px
import plotly.figure_factory as ff

In [0]:
data_covid19 = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"

In [0]:
df_covid = pd.read_csv(data_covid19)


In [6]:
df_covid.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [0]:
# import data for analysis

df_population = pd.read_csv("ACSST1Y2018.S0101_data_with_overlays_2020-04-02T080504.csv")
df_states = pd.read_csv("state-geocodes-v2018.csv")
df_counties = pd.read_csv("all-geocodes-v2018.csv")
df_hospitals = pd.read_csv("Hospitals.csv")

In [75]:
df_population.head()

,GEO_ID,NAME,S0101_C01_001E,S0101_C01_001M,S0101_C02_001E,S0101_C02_001M,S0101_C03_001E,S0101_C03_001M,S0101_C04_001E,S0101_C04_001M,S0101_C05_001E,S0101_C05_001M,S0101_C06_001E,S0101_C06_001M,S0101_C01_002E,S0101_C01_002M,S0101_C02_002E,S0101_C02_002M,S0101_C03_002E,S0101_C03_002M,S0101_C04_002E,S0101_C04_002M,S0101_C05_002E,S0101_C05_002M,S0101_C06_002E,S0101_C06_002M,S0101_C01_003E,S0101_C01_003M,S0101_C02_003E,S0101_C02_003M,S0101_C03_003E,S0101_C03_003M,S0101_C04_003E,S0101_C04_003M,S0101_C05_003E,S0101_C05_003M,S0101_C06_003E,S0101_C06_003M,S0101_C01_004E,S0101_C01_004M,...,S0101_C05_035E,S0101_C05_035M,S0101_C06_035E,S0101_C06_035M,S0101_C01_036E,S0101_C01_036M,S0101_C02_036E,S0101_C02_036M,S0101_C03_036E,S0101_C03_036M,S0101_C04_036E,S0101_C04_036M,S0101_C05_036E,S0101_C05_036M,S0101_C06_036E,S0101_C06_036M,S0101_C01_037E,S0101_C01_037M,S0101_C02_037E,S0101_C02_037M,S0101_C03_037E,S0101_C03_037M,S0101_C04_037E,S0101_C04_037M,S0101_C05_037E,S0101_C05_037M,S0101_C06_037E,S0101_C06_037M,S0101_C01_038E,S0101_C01_038M,S0101_C02_038E,S0101_C02_038M,S0101_C03_038E,S0101_C03_038M,S0101_C04_038E,S0101_C04_038M,S0101_C05_038E,S0101_C05_038M,S0101_C06_038E,S0101_C06_038M
0,id,Geographic Area Name,Estimate!!Total!!Total population,Margin of Error!!Total MOE!!Total population,Estimate!!Percent!!Total population,Margin of Error!!Percent MOE!!Total population,Estimate!!Male!!Total population,Margin of Error!!Male MOE!!Total population,Estimate!!Percent Male!!Total population,Margin of Error!!Percent Male MOE!!Total popul...,Estimate!!Female!!Total population,Margin of Error!!Female MOE!!Total population,Estimate!!Percent Female!!Total population,Margin of Error!!Percent Female MOE!!Total pop...,Estimate!!Total!!Total population!!AGE!!Under ...,Margin of Error!!Total MOE!!Total population!!...,Estimate!!Percent!!Total population!!AGE!!Unde...,Margin of Error!!Percent MOE!!Total population...,Estimate!!Male!!Total population!!AGE!!Under 5...,Margin of Error!!Male MOE!!Total population!!A...,Estimate!!Percent Male!!Total population!!AGE!...,Margin of Error!!Percent Male MOE!!Total popul...,Estimate!!Female!!Total population!!AGE!!Under...,Margin of Error!!Female MOE!!Total population!...,Estimate!!Percent Female!!Total population!!AG...,Margin of Error!!Percent Female MOE!!Total pop...,Estimate!!Total!!Total population!!AGE!!5 to 9...,Margin of Error!!Total MOE!!Total population!!...,Estimate!!Percent!!Total population!!AGE!!5 to...,Margin of Error!!Percent MOE!!Total population...,Estimate!!Male!!Total population!!AGE!!5 to 9 ...,Margin of Error!!Male MOE!!Total population!!A...,Estimate!!Percent Male!!Total population!!AGE!...,Margin of Error!!Percent Male MOE!!Total popul...,Estimate!!Female!!Total population!!AGE!!5 to ...,Margin of Error!!Female MOE!!Total population!...,Estimate!!Percent Female!!Total population!!AG...,Margin of Error!!Percent Female MOE!!Total pop...,Estimate!!Total!!Total population!!AGE!!10 to ...,Margin of Error!!Total MOE!!Total population!!...,...,Estimate!!Female!!Total population!!SUMMARY IN...,Margin of Error!!Female MOE!!Total population!...,Estimate!!Percent Female!!Total population!!SU...,Margin of Error!!Percent Female MOE!!Total pop...,Estimate!!Total!!Total population!!SUMMARY IND...,Margin of Error!!Total MOE!!Total population!!...,Estimate!!Percent!!Total population!!SUMMARY I...,Margin of Error!!Percent MOE!!Total population...,Estimate!!Male!!Total population!!SUMMARY INDI...,Margin of Error!!Male MOE!!Total population!!S...,Estimate!!Percent Male!!Total population!!SUMM...,Margin of Error!!Percent Male MOE!!Total popul...,Estimate!!Female!!Total population!!SUMMARY IN...,Margin of Error!!Female MOE!!Total population!...,Estimate!!Percent Female!!Total population!!SU...,Margin of Error!!Percent Female MOE!!Total pop...,Estimate!!Total!!PERCENT ALLOCATED!!Sex,Margin of Error!!Total MOE!!PERCENT ALLOCATED!...,Estimate!!Percent!!PERCENT ALLOCATED!!Sex,Margin of Error!!Percent MOE!!PERCENT ALLOCATE.

In [0]:
# make the first row (Row 0) the column header names

df_population.columns = df_population.iloc[0]

In [0]:
# remove the first row 
#(i.e. make the dataframe now all of the data after the first row)

df_population = df_population[1:]

In [78]:
# look at all columns in dataframe
df_population.columns.tolist()

['id',
 'Geographic Area Name',
 'Estimate!!Total!!Total population',
 'Margin of Error!!Total MOE!!Total population',
 'Estimate!!Percent!!Total population',
 'Margin of Error!!Percent MOE!!Total population',
 'Estimate!!Male!!Total population',
 'Margin of Error!!Male MOE!!Total population',
 'Estimate!!Percent Male!!Total population',
 'Margin of Error!!Percent Male MOE!!Total population',
 'Estimate!!Female!!Total population',
 'Margin of Error!!Female MOE!!Total population',
 'Estimate!!Percent Female!!Total population',
 'Margin of Error!!Percent Female MOE!!Total population',
 'Estimate!!Total!!Total population!!AGE!!Under 5 years',
 'Margin of Error!!Total MOE!!Total population!!AGE!!Under 5 years',
 'Estimate!!Percent!!Total population!!AGE!!Under 5 years',
 'Margin of Error!!Percent MOE!!Total population!!AGE!!Under 5 years',
 'Estimate!!Male!!Total population!!AGE!!Under 5 years',
 'Margin of Error!!Male MOE!!Total population!!AGE!!Under 5 years',
 'Estimate!!Percent Male!!T

In [0]:
# make a list of all columns that start with "Margin of Error"

margin_error = df_population.columns.str.startswith("Margin of Error!!")

In [0]:
# keeep only NOT true values for margin of error
df_pop_values = df_population.loc[:, ~margin_error]

In [0]:
# remove columns that have a (X) value 

df_us_pop = df_pop_values.drop(columns=df_pop_values.columns[(df_pop_values == '(X)').any()])

In [0]:
# strip "Estimate!!" from columns because it's on the outside of the name
df_us_pop.columns = df_us_pop.columns.str.strip("Estimate!!")

In [0]:
# remove !!Total population from column headers by replacing with nothing
df_us_pop.columns = df_us_pop.columns.str.replace("!!Total population", "")

In [0]:
# make list of columns we want to keep for analysis

uspop_col = ['d',
             'Geographic Area N',
             'Total',
             'Male',
             'Female',
             'Total!!SELECTED AGE CATEGORIES!!Under 18 year',
             'Male!!SELECTED AGE CATEGORIES!!Under 18 year',
             'Female!!SELECTED AGE CATEGORIES!!Under 18 year',
             'Total!!SELECTED AGE CATEGORIES!!60 years and over',
             'Male!!SELECTED AGE CATEGORIES!!60 years and over',
             'Female!!SELECTED AGE CATEGORIES!!60 years and over',
             'Total!!SUMMARY INDICATORS!!Median age (years)',
             'Total!!SUMMARY INDICATORS!!Sex ratio (males per 100 females)',
             'Total!!SUMMARY INDICATORS!!Age dependency ratio',
             'Total!!SUMMARY INDICATORS!!Old-age dependency ratio',
             'Total!!SUMMARY INDICATORS!!Child dependency ratio',
            ]

In [0]:
# make new dataframe with only new column subset
df_us_pop_filter = df_us_pop.filter(items = uspop_col)

In [0]:
# remove "!!SELECTED AGE CATEGORIES!!" from column headers
df_us_pop_filter.columns = df_us_pop_filter.columns.str.replace("!!SELECTED AGE CATEGORIES!!", " ")

# remove "TOTAL!!!SUMMARY INDICATORS!!" from column headers
df_us_pop_filter.columns = df_us_pop_filter.columns.str.replace("Total!!SUMMARY INDICATORS!!", "")

In [0]:
# re-import state data
df_states = pd.read_csv("state-geocodes-v2018.csv", skiprows = 5)

In [0]:
# re-import county data
df_county = pd.read_csv("all-geocodes-v2018.csv", skiprows = 4)

In [0]:
# remove if county code is 0 (entire united states)

df_county = df_county[df_county["County Code (FIPS)"] != 0]

In [0]:
# convert state and county fips columns to strings

df_county[["State Code (FIPS)", "County Code (FIPS)"]] = df_county[["State Code (FIPS)", "County Code (FIPS)"]].astype(str)

In [0]:
# add leading zeros for a two digit value in state fips
df_county["State Code (FIPS)"] = df_county["State Code (FIPS)"].str.zfill(2)

# add leading zeros for three digit value in county fips
df_county["County Code (FIPS)"] = df_county["County Code (FIPS)"].str.zfill(3)

In [0]:
# make full county fips number
df_county["full_county_fips"] = df_county["State Code (FIPS)"] + df_county["County Code (FIPS)"]

In [0]:
# convert state fips code to string
df_states["State (FIPS)"] = df_states["State (FIPS)"].astype(str)

# add preceding zeros to make 2-digit FIPS codes
df_states["State (FIPS)"] = df_states["State (FIPS)"].str.zfill(2)

In [0]:
# merge state name onto county fips dataframe
df_county = pd.merge(df_county, 
                     df_states[["State (FIPS)", "Name"]], 
                     how = "left", 
                     left_on = "State Code (FIPS)", 
                     right_on = "State (FIPS)")

In [0]:
# create column of county state and name
df_county["county_state_name"] = df_county["Area Name (including legal/statistical area description)"] + ", " + df_county["Name"]

In [0]:
# merge full county fips name and number to population dataframe
df_county_pop = pd.merge(df_us_pop_filter,
                         df_county[["county_state_name", "full_county_fips"]],
                         how = "left",
                         left_on = "Geographic Area N",
                         right_on = "county_state_name"
                        )

In [96]:
df_county_pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 827 entries, 0 to 837
Data columns (total 20 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   d                                  827 non-null    object 
 1   Geographic Area N                  827 non-null    object 
 2   Total                              827 non-null    int64  
 3   Male                               827 non-null    int64  
 4   Female                             827 non-null    int64  
 5   Total Under 18 year                827 non-null    int64  
 6   Male Under 18 year                 827 non-null    int64  
 7   Female Under 18 year               827 non-null    int64  
 8   Total 60 years and over            827 non-null    int64  
 9   Male 60 years and over             827 non-null    int64  
 10  Female 60 years and over           827 non-null    int64  
 11  Median age (years)                 827 non-null    float64

In [0]:
# remove rows that have null values in the county, state name and fips number
df_county_pop = df_county_pop[df_county_pop["full_county_fips"].notnull()]

In [0]:
# make list of columns to convert to integers to use in calculations later

int_list = [
 'Total',
 'Male',
 'Female',
 'Total Under 18 year',
 'Male Under 18 year',
 'Female Under 18 year',
 'Total 60 years and over',
 'Male 60 years and over',
 'Female 60 years and over']

In [0]:
# columns to convert to floats
float_list = [ 
    'Median age (years)',
    'Sex ratio (males per 100 females)',
    'Age dependency ratio',
    'Old-age dependency ratio',
    'Child dependency ratio']

In [0]:
# convert columns in int_list to integers
df_county_pop[int_list] = df_county_pop[int_list].astype(int)
df_county_pop[float_list] = df_county_pop[float_list].astype(float)

In [0]:
# percentage of pop under 18 yo
df_county_pop["percent_under18"] = (df_county_pop["Total Under 18 year"]/df_county_pop["Total"])*100

In [0]:
# percentage of county residents over 60 years old
df_county_pop["percent_over60"] = (df_county_pop["Total 60 years and over"]/df_county_pop["Total"])*100

In [0]:
# remove values in hospital df with "NOT AVAILABLE" values

df_hospitals= df_hospitals[df_hospitals["COUNTYFIPS"] != "NOT AVAILABLE"]

In [0]:
# remove nulls from countyfips column
df_hospitals = df_hospitals[df_hospitals["COUNTYFIPS"].notnull()]

In [0]:
# remove if beds is -999
df_county_hospitals = df_hospitals[df_hospitals["BEDS"] != -999]

In [0]:
# calculate number of hospitals, beds per county
# create an aggregated table for these values

df_county_hospitals = df_hospitals.groupby("COUNTYFIPS").agg({"BEDS": "sum", 
                                                                "NAME": "count"}).reset_index()

In [0]:
# rename "NAME" to hospital_count and "BEDS" to hospital_bed_count

df_county_hospitals = df_county_hospitals.rename(columns = {"NAME": "hospital_count",
                                                           "BEDS": "hospital_bed_count"})

In [0]:
# merge hospital data onto county population data

df_county_health = pd.merge(df_county_pop,
                           df_county_hospitals,
                           how = "left",
                           left_on = "full_county_fips",
                           right_on = "COUNTYFIPS")

In [0]:
# we're missing 10 county matches, so we'll delete these for now
df_county_health = df_county_health[df_county_health["COUNTYFIPS"].notnull()]

In [0]:
# export county health information to csv
df_county_health.to_csv("us_county_population_hospital_data_2018.csv")

In [0]:
# sort county data by 18 under population 
df_total_pop = df_county_health.sort_values(by = "Total", ascending = False)

In [0]:
# top 20 
df_total_top20 = df_total_pop.head(20)

In [0]:
# make a graph to show percent of largest 18 under population 
bar_total_pop = px.bar(df_total_top20,
                   x = "Geographic Area N",
                   y = "Total",
                   title = "US Counties with the Highest Total Population",
                   labels = {"Geographic Area N": "County, State Name", "Total": "Total Population"})

In [114]:
bar_total_pop

In [115]:
bubble_u18_o60 = px.scatter(df_county_health,
                            x = "percent_over60",
                            y = "percent_under18",
                            size = "hospital_bed_count",
                            hover_data = ['Geographic Area N',"Total"],
                            title = "Percentage of Population Under 18 and Over 60 in US Counties",
                            labels = {"Geographic Area N": "County, State Name",
                                      "Total": "Total Population",
                                      "percent_over60": "Percent of County Population over 60 years old",
                                      "percent_under18": "Percent of County Population under 18 years old"})

ValueError: ignored

In [0]:
# make a column to standardize hospital bed count
# number of hospital beds per 1,000 people
df_county_health["hospitalbeds_per10000"] = (df_county_health["hospital_bed_count"]/df_county_health["Total"])*1000

In [116]:
# make a graph to show percent of largest 18 under population 
bubble_u18_o60 = px.scatter(df_county_health,
                            x = "percent_over60",
                            y = "percent_under18",
                            size = "hospital_bed_count",
                            hover_data = ['Geographic Area N',"Total"],
                            title = "Percentage of Population Under 18 and Over 60 in US Counties",
                            labels = {"Geographic Area N": "County, State Name",
                                      "Total": "Total Population",
                                      "percent_over60": "Percent of County Population over 60 years old",
                                      "percent_under18": "Percent of County Population under 18 years old"})

ValueError: ignored

In [0]:
# create a new dataframe that only includes the most recent coronavirus data
df_covid_apr10 = df_covid[df_covid["date"]== "2020-04-10"]

In [0]:
# first, remove null values from fips
df_covid_apr10 = df_covid_apr10[df_covid_apr10["fips"].notnull()]

# convert fips number to int and then string
df_covid_apr10["fips"] = df_covid_apr10["fips"].astype(int)
df_covid_apr10["fips"] = df_covid_apr10["fips"].astype(str)

In [0]:
# adding preceding zeros so that the FIPS code includes 5 digits
df_covid_apr10["fips"] = df_covid_apr10["fips"].str.zfill(5)

In [0]:
# merge covid data to df_county_health data
df_county_health = pd.merge(df_county_health,
                           df_covid_apr10[["fips", "cases", "deaths"]],
                           how = "left",
                           left_on = "full_county_fips",
                           right_on = "fips")

In [0]:
# make two more columns that show number of cases/deaths per 1000 people
df_county_health["cases_per1000"] = (df_county_health["cases"]/df_county_health["Total"])*1000
df_county_health["deaths_per1000"] = (df_county_health["deaths"]/df_county_health["Total"])*1000

In [0]:
# correlation table
covid_corr = df_county_health.corr()

In [0]:
# make an annotated heatmap
county_covid19_heatmap = ff.create_annotated_heatmap(z = covid_corr.values, 
                                       x=list(covid_corr.columns), 
                                       y=list(covid_corr.index), 
                                       annotation_text=covid_corr.round(2).values, 
                                       showscale = True 
                                       )

In [69]:
county_covid19_heatmap
